In [1]:
import sklearn
from torch2trt import TRTModule

In [2]:
import json
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
import trt_pose.coco
import math
import os
import numpy as np
import traitlets
import pickle 


In [3]:
with open('preprocess/human_pose.json', 'r') as f:
    hand_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(hand_pose)
import trt_pose.models

num_parts = len(hand_pose['keypoints'])
num_links = len(hand_pose['skeleton'])

model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()
import torch


WIDTH = 224
HEIGHT = 224
data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()

if not os.path.exists('model/resnet18_baseline_att_224x224_A_epoch_249_trt.pth'):
    MODEL_WEIGHTS = 'model/resnet18_baseline_att_224x224_A_epoch_249.pth'
    model.load_state_dict(torch.load(MODEL_WEIGHTS))
    import torch2trt
    model_trt = torch2trt.torch2trt(model, [data], fp16_mode=True, max_workspace_size=1<<25)
    OPTIMIZED_MODEL = 'model/resnet18_baseline_att_224x224_A_epoch_249_trt.pth'
    torch.save(model_trt.state_dict(), OPTIMIZED_MODEL)


OPTIMIZED_MODEL = 'model/resnet18_baseline_att_224x224_A_epoch_249_trt.pth'


model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

[05/04/2022-23:36:33] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.


<All keys matched successfully>

In [4]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology,cmap_threshold=0.12, link_threshold=0.15)
draw_objects = DrawObjects(topology)

In [5]:

import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto', kernel='rbf'))

In [7]:
from preprocessdata import preprocessdata
preprocessdata = preprocessdata(topology, num_parts)

In [8]:
svm_train = False
if svm_train:
    clf, predicted = preprocessdata.trainsvm(clf, joints_train, joints_test, hand.labels_train, hand.labels_test)
    filename = 'svmmodel.sav'
    pickle.dump(clf, open(filename, 'wb'))
else:
    filename = 'svmmodel_new.sav'
    clf = pickle.load(open(filename, 'rb'))

In [9]:
with open('preprocess/gesture_new.json', 'r') as f:
    gesture = json.load(f)
gesture_type = gesture["classes"]


In [10]:
from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30, capture_device=0)
#camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=30)

camera.running = True

[ WARN:0] global /home/ubuntu/build_opencv/opencv/modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


In [11]:
def draw_joints(image, joints):
    count = 0
    for i in joints:
        if i==[0,0]:
            count+=1
    if count>= 3:
        return 
    for i in joints:
        cv2.circle(image, (i[0],i[1]), 2, (0,0,255), 1)
    cv2.circle(image, (joints[0][0],joints[0][1]), 2, (255,0,255), 1)
    for i in hand_pose['skeleton']:
        if joints[i[0]-1][0]==0 or joints[i[1]-1][0] == 0:
            break
        cv2.line(image, (joints[i[0]-1][0],joints[i[0]-1][1]), (joints[i[1]-1][0],joints[i[1]-1][1]), (0,255,0), 1)

In [12]:
import ipywidgets
from IPython.display import display


image_w = ipywidgets.Image(format='jpeg', width=224, height=224)
display(image_w)

Image(value=b'', format='jpeg', height='224', width='224')

In [13]:
from pynput.keyboard import Key, Controller

keyboard = Controller()

BUTTON = ('punch', 'kick', 'up')
JOYSTICK = ('left', 'right', 'down')

keys_dict = {
    'punch': 'j',
    'kick': 'k',
    'left': 'a',
    'right': 'd',
    'up': 'w',
    'down': 's',
#     'stand': '.'
}

key_pressed = []
last_text = ''

import time

def action(text):
    global last_text
    if text == last_text:
        return
    last_text = text
    
    if text in BUTTON:
        key = keys_dict.get(text)
        keyboard.press(key)
        print('===========short pressed', key)
        time.sleep(0.15)
        keyboard.release(key)
    elif text in JOYSTICK:
        key = keys_dict.get(text)
        if key not in key_pressed:
            for k in key_pressed:
                try:
                    keyboard.release(k)
                except:
                    pass
            key_pressed.clear()
            keyboard.press(key)
            print('============long pressed', key)
            key_pressed.append(key)
    elif text == 'act':
        print('===========do action!')
        keyboard.press('d')
        time.sleep(0.05)
        keyboard.press('s')
        time.sleep(0.05)
        keyboard.release('d')
        keyboard.press('a')
        time.sleep(0.05)
        keyboard.release('s')
        time.sleep(0.05)
        keyboard.release('a')
        keyboard.press('j')
        time.sleep(0.05)
        keyboard.release('j')
    else:
        for k in key_pressed:
            try:
                keyboard.release(k)
            except:
                pass
        key_pressed.clear()
#     if key_pressed:
#         print(key_pressed)

In [14]:
def execute(change):
    image = change['new']
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)
    joints = preprocessdata.joints_inference(image, counts, objects, peaks)
    draw_joints(image, joints)
    #draw_objects(image, counts, objects, peaks)
    dist_bn_joints = preprocessdata.find_distance(joints)
    gesture = clf.predict([dist_bn_joints,[0]*num_parts*num_parts])
    gesture_joints = gesture[0]
    preprocessdata.prev_queue.append(gesture_joints)
    preprocessdata.prev_queue.pop(0)
    _, text = preprocessdata.print_label(image, preprocessdata.prev_queue, gesture_type)
    action(text)  # send key event
    image_w.value = bgr8_to_jpeg(image)
    
#     if text:
#         print(text)
    


In [15]:
execute({'new': camera.value})

In [16]:
camera.observe(execute, names='value')

In [17]:
# camera.unobserve_all()

In [18]:
# camera.running = False

============long pressed a
============long pressed a
===========short pressed j
===========short pressed j
===========short pressed k
===========short pressed j
===========do action!
===========do action!
============long pressed a
============long pressed a
============long pressed a
============long pressed a
============long pressed a
============long pressed a
============long pressed s
============long pressed a
============long pressed s
============long pressed a
============long pressed s
============long pressed a
============long pressed a
============long pressed a
============long pressed s
============long pressed a
===========do action!
============long pressed a
============long pressed a
===========short pressed j
============long pressed a
============long pressed d
============long pressed a
===========do action!
============long pressed a
===========do action!
===========short pressed k
============long pressed a
============long pressed a
============long pressed a